## **HR ANALYTICS**

Download employee retention dataset from here: https://www.kaggle.com/giripujar/hr-analytics.

Now do some exploratory data analysis to figure out which variables have direct and clear impact on employee retention (i.e. whether they leave the company or continue to work) <br>
Plot bar charts showing impact of employee salaries on retention <br>
Plot bar charts showing corelation between department and employee retention <br>
Now build logistic regression model using variables that were narrowed down in step 1 <br>
Measure the accuracy of the model